In [ ]:
import numpy as np
import scipy.io
import os
from tqdm.notebook import trange

In [ ]:
# loadScaleList = ['random']
loadScaleList = ['1.00', '1.10', '1.20', '1.30', '1.40', '1.50', '1.60', '1.70', '1.80', '1.90', '2.00']
N_nodes = 89
numSamples = 10000
processCascadeData = True
processInfluenceModelWeights = True

caseName = 'IEEE' + str(N_nodes)
numSamplesFolderName = 'samples' + str(numSamples) + '/'
numSamplesTrain = int(np.round(0.9*numSamples))
numSamplesTest = numSamples - numSamplesTrain
M_linesDict = {89:206,118:179,1354:1710}
M_lines = M_linesDict[N_nodes]
numNodeFeatures = 2
numEdgeFeatures = 2

for loadScale in loadScaleList:
    dataDirectory = './data/'
    dataDirectory = dataDirectory + 'dc/'
    dataDirectory = dataDirectory + caseName + '/'
    linksetdataDirectory = dataDirectory

    dataDirectory = dataDirectory + 'load' + loadScale + 'gen' + loadScale + '/'
    dataDirectory = dataDirectory + numSamplesFolderName
    print(dataDirectory)
    if(processCascadeData): prepareOneShotData()
    if(processInfluenceModelWeights): prepareInfluenceModelData()

In [ ]:
def readVariablePairs(fileName, variable1, variable2):
    variable1 = scipy.io.loadmat(fileName)[variable1].T
    variable2 = scipy.io.loadmat(fileName)[variable2].T
    currData = np.stack((variable1, variable2), axis=-1)
    return currData

In [ ]:
def readOneShotData():
    nodeData = np.zeros((numSamples, N_nodes, numNodeFeatures))
    edgeData = np.zeros((numSamples, M_lines, numEdgeFeatures))
    for ii in trange(numSamples):
        fileName =  dataDirectory + 'data_sample' + str(ii) + '.mat'
        
        # read node data
        currNodeData = readVariablePairs(fileName, 'powerConsumeData', 'powerSuppliedData')
        nodeData[ii,:,:] = currNodeData[0,:,:]

        # read edge data
        currEdgeData = readVariablePairs(fileName, 'activeLinesData', 'powerFlowData')

        edgeData[ii,:,0] = currEdgeData[0,:,0]

        a = np.sum(currEdgeData[:,:,0], axis=0)
        a[a == np.max(a)] = -1
        edgeData[ii,:,1] = a
        
    return nodeData, edgeData

In [ ]:
def prepareOneShotData():
    # read node edge data
    nodeData, edgeData = readOneShotData()

    nodeDataTrain = nodeData[:numSamplesTrain,:,:]
    edgeDataTrain = edgeData[:numSamplesTrain,:,:]
    nodeDataTest = nodeData[numSamplesTrain:,:,:]
    edgeDataTest = edgeData[numSamplesTrain:,:,:]

    print(nodeDataTrain.shape)
    print(edgeDataTrain.shape)
    print(nodeDataTest.shape)
    print(edgeDataTest.shape)

    # read linkset data
    linksetFileLocation = linksetdataDirectory  + 'linksetAndCapacity' + str(caseName) 
    linksetFileLocation = linksetFileLocation + 'load' + loadScale + 'gen' + loadScale + '.mat'

    linkSet = scipy.io.loadmat(linksetFileLocation)['linkSet']
    linkSet = linkSet-1
    linkSet = linkSet.astype(int)

    # save prepared data
    baseFolder = '../gnnmodel'
    baseName = baseFolder + '/data/' + caseName + '/' + 'load' + loadScale + 'gen' + loadScale + '/'
    if not os.path.exists(baseFolder):
        print('Base folder did not exist')
    if not os.path.exists(baseName):
        print('New data folder created')
        os.makedirs(baseName)
    print('Saved in folder ' + baseName)

    np.save(baseName + 'nodeDataTrain', nodeDataTrain)
    np.save(baseName + 'edgeDataTrain', edgeDataTrain)

    np.save(baseName + 'nodeDataTest', nodeDataTest)
    np.save(baseName + 'edgeDataTest', edgeDataTest)

    np.save(baseName + 'linkSet', linkSet)

In [ ]:
def prepareInfluenceModelData():
    # read influence data
    influenceModelWeightFiles = dataDirectory + 'influenceModelParams.mat'
    A01 = scipy.io.loadmat(influenceModelWeightFiles)['A01']
    A11 = scipy.io.loadmat(influenceModelWeightFiles)['A11']
    D = scipy.io.loadmat(influenceModelWeightFiles)['D']
    epsilonVals = scipy.io.loadmat(influenceModelWeightFiles)['epsilonVals']
    finalStates = scipy.io.loadmat(influenceModelWeightFiles)['finalStates']
    initialStates = scipy.io.loadmat(influenceModelWeightFiles)['initialStates']

    # save influence data
    baseFolder = '../influencemodel'
    baseName = baseFolder + '/data/' + caseName + '/' + 'load' + loadScale + 'gen' + loadScale + '/'
    if not os.path.exists(baseFolder):
        print('Base folder did not exist')
    if not os.path.exists(baseName):
        print('New data folder created')
        os.makedirs(baseName)
    print('Saved in folder ' + baseName)
    
    np.save(baseName + 'A01', A01)
    np.save(baseName + 'A11', A11)
    np.save(baseName + 'D', D)
    np.save(baseName + 'epsilonVals', epsilonVals)
    np.save(baseName + 'finalStates', finalStates)
    np.save(baseName + 'initialStates', initialStates)